## imports and data urls

In [24]:
import pandas as pd
import numpy as np

url1 = "https://datasets.imdbws.com/title.basics.tsv.gz"
url2 = "https://datasets.imdbws.com/title.akas.tsv.gz"
url3 = "https://datasets.imdbws.com/title.ratings.tsv.gz"

## Lets clean akas

In [25]:
%%time

# Load akas Data
akas = pd.read_csv(url2, sep='\t', low_memory=False)

CPU times: total: 2min 33s
Wall time: 4min 28s


In [26]:
%%time

akas.replace({'\\N':np.nan}, inplace=True)
akas = akas[(akas['region'] == 'US')]
akas.head()

CPU times: total: 1min 18s
Wall time: 1min 36s


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


## Lets clean title.basics dataset

In [27]:
%%time

# Load basics Data
basics = pd.read_csv(url1, sep='\t', low_memory=False)

CPU times: total: 42.7 s
Wall time: 1min 2s


In [28]:
# Replace '\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [29]:
# Drop any rows that have null values in either genre or runtimeMinutes.
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [30]:
# Drop all rows where titleType!=Movie. Keep only titleType==Movie
is_Movie = basics['titleType'] == 'movie'
basics = basics[is_Movie]

In [31]:
# Drop all rows where the start year is not between 2000-2022. Keep startYear 2000-2022
basics.info()

# We first need to change the column type to something numeric.
print("\nData Type before conversion : ", basics['startYear'].dtype)
basics['startYear'] = pd.to_numeric(basics['startYear'])
print("\nData Type after conversion : ", basics['startYear'].dtype)

#Now we can make and apply our filters
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369491 entries, 8 to 9289365
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          369491 non-null  object
 1   titleType       369491 non-null  object
 2   primaryTitle    369491 non-null  object
 3   originalTitle   369491 non-null  object
 4   isAdult         369491 non-null  object
 5   startYear       363574 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  369491 non-null  object
 8   genres          369491 non-null  object
dtypes: object(9)
memory usage: 28.2+ MB
Data Type before conversion :  object

Data Type after conversion :  float64


In [32]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [33]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers_basics = basics['tconst'].isin(akas['titleId'])

basics = basics[keepers_basics]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86770,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93906,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


## Lets clean ratings

In [34]:
%%time

# Load ratings Data
ratings = pd.read_csv(url3, sep='\t', low_memory=False)

CPU times: total: 1.41 s
Wall time: 2.56 s


In [35]:
ratings.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1917
1,tt0000002,5.8,260
4,tt0000005,6.2,2541
5,tt0000006,5.1,175
6,tt0000007,5.4,796


## save cleaned files in Data folder

In [36]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)